<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/37_PyTorch_DataLoader_Source_Code_Debugging_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch DataLoader Source Code - Debugging Session

在这节中，我们将继续上一次关于数据规范化的讨论。只是这一次，我们不是在编写代码，而是在调试代码，具体来说，我们将在PyTorch源代码中进行调试，以查看在规范化数据集时到底发生了什么。

## 1.调试PyTorch源代码的简短程序

在开始调试之前，我们只想简要介绍一下我们编写的程序，可以使我们介入并查看数据集的规范化，并确切了解如何在PyTorch之下完成该操作

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader

正如我们在上一集中讨论的那样，我们具有均值和标准差值。 现在，不必计算它们，我们只需要提取它们并将它们硬编码到此处的程序中即可。

In [ ]:
mean = 0.2860347330570221
std = 0.3530242443084717

如果要脱机获取这些值，我们将做这种事情。

我们不想麻烦重新计算这些值，因此我们在这里很难对其进行描述。 我们有均值和标准差，我们知道我们需要这两个值才能对数据集的每个成员或每个像素进行归一化。

接下来，我们使用FashionMNIST类构造函数初始化训练集。 这里要注意或要注意的关键是transforms。 我们有transforms的组成

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train = True,
    download = True,
    transform =transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
)


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


第一个Composition将图像转换为张量，然后第二个Composition是规格化变换，它将对我们的数据进行规格化。我们的目标是在源代码中验证这个特定转换是如何工作的。

最后，我们创建一个DataLoader并使用它。

In [ ]:
loader = DataLoader(train_set,batch_size=1)
image,label = next(iter(loader))

## 2.调试PyTorch源代码
好的，现在我们可以进行实际调试了。 要进行调试，我们将继续进行，仅确保选择了我的python运行配置，然后单击，开始调试。

使用此[链接](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataloader.html#DataLoader)可以访问PyTorch DataLoader类的当前源代码。 本讨论假设使用PyTorch版本1.5.0。

### The Sampler: To Shuffle Or Not
---
采样器是获取索引值的对象，该索引值将用于从基础数据集中获取实际值。

我们可以看到，有两个相关的特定采样器，随机采样器和顺序采样器。

* 随机取样器

* 顺序采样器

如果混洗值是true，则采样器将是随机采样器，否则将是连续采样器

### 批次大小的使用方式
---

我们发现采样器用于在以下代码中收集索引值：

In [ ]:
def __iter__(self):
    batch = []
    for idx in self.sampler:
        batch.append(idx)
        if len(batch) == self.batch_size:
            yield batch
            batch = []
    if len(batch) > 0 and not self.drop_last:
        yield batch

在这里，我们可以看到batch_size参数在起作用，因为它限制了所收集索引值的数量。

请注意，此处的yield关键字使此迭代器成为所谓的生成器。

获取索引值后，它们将通过以下方式用于获取数据：

In [ ]:
def fetch(self, possibly_batched_index):
    if self.auto_collation:
        data = [self.dataset[idx] for idx in possibly_batched_index]
    else:
        data = self.dataset[possibly_batched_index]
    return self.collate_fn(data)    

这样做的工作是从底层数据集中提取每个样本。

```
data = [self.dataset[idx] for idx in possibly_batched_index]
```


这将返回一个数据元素列表，然后使用collat​​e_fn（）方法将其提取并放入单个批处理张量中。



### 标准化数据集
---
最后，我们发现使用function api的normalize（）函数对返回到批处理中的每个元素进行了规范化。

```
def normalize(tensor, mean, std, inplace=False):
    """Normalize a tensor image with mean and standard deviation.
    tensor.sub_(mean).div_(std)
    return tensor

```
注意，dataset类调用一个transforms，然后调用函数api。我们也遇到了一些糟糕的设计，这需要一些hacking来保持事情的一致性。

注意，在这里使用术语hacking，我们指的是这样一个事实：我们看到代码正在进行不必要的转换。
